In [ ]:
from ase.atoms import Atoms
from ase.build import bulk as _bulk

In [ ]:
from atomistics.workflows import (
    optimize_positions_and_volume as _optimize_positions_and_volume,
)
from atomistics.workflows.evcurve.helper import (
    analyse_structures_helper,
    generate_structures_helper,
)
from atomistics.calculators import (
    evaluate_with_lammpslib as _evaluate_with_lammpslib, 
    get_potential_by_name as _get_potential_by_name,
)

In [ ]:
import pandas

In [ ]:
from pyiron_workflow import as_function_node, Workflow

In [ ]:
@as_function_node
def bulk(name: str, cubic: bool = False) -> Atoms:
    return _bulk(name=name, cubic=cubic)

In [ ]:
@as_function_node
def get_potential_by_name(potential_name: str) -> pandas.Series:
    return _get_potential_by_name(potential_name=potential_name)

In [ ]:
@as_function_node
def optimize_positions_and_volume(structure: Atoms) -> dict:
    return _optimize_positions_and_volume(structure=structure)

In [ ]:
@as_function_node
def evaluate_with_lammpslib(task_dict: dict, potential_dataframe: pandas.Series) -> dict:
    return _evaluate_with_lammpslib(task_dict=task_dict, potential_dataframe=potential_dataframe)

In [ ]:
@as_function_node
def evcurve_generate_structures(structure: dict, vol_range: float, num_points: int) -> dict:
    return {
        "calc_energy": generate_structures_helper(
            structure=structure["structure_with_optimized_positions_and_volume"],
            vol_range=vol_range,
            num_points=num_points,
        ),
    }

In [ ]:
@as_function_node
def evcurve_analyse_structures(output_dict: dict, structure_dict: dict, fit_type: str = "polynomial", fit_order: int = 3) -> dict:
    return analyse_structures_helper(
        output_dict=output_dict,
        structure_dict=structure_dict["calc_energy"],
        fit_type=fit_type,
        fit_order=fit_order,
    )

In [ ]:
wf = Workflow(label="evcurve")

In [ ]:
wf.structure = bulk("Al", cubic=True)

In [ ]:
wf.df_pot_selected = get_potential_by_name(
    potential_name="1999--Mishin-Y--Al--LAMMPS--ipr1",
)

In [ ]:
wf.opt_task_dict = optimize_positions_and_volume(structure=wf.structure)

In [ ]:
wf.opt_result_dict = evaluate_with_lammpslib(
    task_dict=wf.opt_task_dict,
    potential_dataframe=wf.df_pot_selected,
)

In [ ]:
wf.structure_dict = evcurve_generate_structures(
    structure=wf.opt_result_dict,
    vol_range=0.05,
    num_points=11,
)

In [ ]:
wf.result_dict = evaluate_with_lammpslib(
    task_dict=wf.structure_dict,
    potential_dataframe=wf.df_pot_selected,
)

In [ ]:
wf.fit_dict = evcurve_analyse_structures(
    output_dict=wf.result_dict,
    structure_dict=wf.structure_dict,
    fit_type="polynomial",
    fit_order=3,
)
wf.fit_dict.pull()

In [ ]:
wf.draw(size=(10,10))